In [1]:
%autosave 0

Autosave disabled


In [45]:
import pandas as pd
import numpy as np
from sqlalchemy import text, create_engine

from env import get_connection

We can create dataframes from..
- Dictionaries
- SQL queries!

In [3]:
pd.DataFrame({'name':['adam','edwige','marc','theo'],
              'eyewear':[True,False,True,True]})

,name,eyewear
0,adam,True
1,edwige,False
2,marc,True
3,theo,True


In [47]:
url = get_connection('farmers_market')
engine = create_engine(url)

In [8]:
query = text('''
        SELECT * FROM customer_purchases
        ''')

In [16]:
df1 = pd.read_sql(query,engine.connect())
df1.head()

,product_id,vendor_id,market_date,customer_id,quantity,cost_to_customer_per_qty,transaction_time
0,1,7,2019-07-03,14,0.99,6.99,0 days 17:32:00
1,1,7,2019-07-03,14,2.18,6.99,0 days 18:23:00
2,1,7,2019-07-03,15,1.53,6.99,0 days 18:41:00
3,1,7,2019-07-03,16,2.02,6.99,0 days 18:18:00
4,1,7,2019-07-03,22,0.66,6.99,0 days 17:34:00


In [14]:
pd.read_sql( (text('SHOW TABLES')) , engine.connect() )

,Tables_in_farmers_market
0,booth
1,customer
2,customer_purchases
3,market_date_info
4,product
5,product_category
6,vendor
7,vendor_booth_assignments
8,vendor_inventory
9,zip_data


In order to run a SQL query against Codeup's database, we need a number of things:
- A URL with our credentials and desired database
- A SQLAlchemy engine object created on the URL
- A SQL query as a text object
- Run pd.read_sql() with the query and the engine.connect()

We will use the farmers_market database on Codeup's SQL server.

We will run the following query:

SELECT *  

FROM customer_purchases  

INNER JOIN product ON customer_purchases.product_id = product.product_id

In [15]:
query2 = text('''
        SELECT * FROM customer_purchases
        INNER JOIN product ON customer_purchases.product_id = product.product_id
        ''')

In [21]:
df = pd.read_sql(query2,engine.connect())
df.head(3)

,product_id,vendor_id,market_date,customer_id,quantity,cost_to_customer_per_qty,transaction_time,product_id,product_name,product_size,product_category_id,product_qty_type
0,1,7,2019-07-03,14,0.99,6.99,0 days 17:32:00,1,Habanero Peppers - Organic,medium,1,lbs
1,1,7,2019-07-03,14,2.18,6.99,0 days 18:23:00,1,Habanero Peppers - Organic,medium,1,lbs
2,1,7,2019-07-03,15,1.53,6.99,0 days 18:41:00,1,Habanero Peppers - Organic,medium,1,lbs


Boolean masking will not make a triumphant return but it's still relevant!

We can subset our dataframe using .loc and .iloc:
- .loc will use labels to access specific rows/columns
- .iloc will use index positions to access specific rows/columns

In [25]:
df.loc[:,'quantity':'product_size'].head()

,quantity,cost_to_customer_per_qty,transaction_time,product_id,product_name,product_size
0,0.99,6.99,0 days 17:32:00,1,Habanero Peppers - Organic,medium
1,2.18,6.99,0 days 18:23:00,1,Habanero Peppers - Organic,medium
2,1.53,6.99,0 days 18:41:00,1,Habanero Peppers - Organic,medium
3,2.02,6.99,0 days 18:18:00,1,Habanero Peppers - Organic,medium
4,0.66,6.99,0 days 17:34:00,1,Habanero Peppers - Organic,medium


In [27]:
df.iloc[0:5,0:5]

,product_id,vendor_id,market_date,customer_id,quantity
0,1,7,2019-07-03,14,0.99
1,1,7,2019-07-03,14,2.18
2,1,7,2019-07-03,15,1.53
3,1,7,2019-07-03,16,2.02
4,1,7,2019-07-03,22,0.66


We can use the .agg() method to perform a specified aggregate function on one column!

We can pass in a list of columns to aggregate and/or perform a list of aggregate functions!

In [29]:
df.quantity.agg(['mean','max','min'])

mean     2.858112
max     20.000000
min      0.020000
Name: quantity, dtype: float64

In [33]:
df['total_cost'] = (df['cost_to_customer_per_qty'] * df['quantity'])

We can use the .groupby() method to consider the unique values in a column.

We then specify a second column on which to perform some aggregate calculations!

In [34]:
df[['quantity','total_cost']].agg(['mean','max','min'])

,quantity,total_cost
mean,2.858112,16.400648
max,20.000000,144.000000
min,0.020000,0.069800


In [37]:
df.groupby('product_name')['total_cost'].agg('mean')

product_name
Apple Pie                     31.404930
Banana Peppers - Jar          11.890672
Cherry Pie                    31.867826
Habanero Peppers - Organic    16.120788
Jalapeno Peppers - Organic    10.749224
Poblano Peppers - Organic      1.591040
Sweet Corn                     2.540638
Whole Wheat Bread             16.962217
Name: total_cost, dtype: float64

The .transform() method returns an aggregate calculation, where the result is the same length as the dataframe!

In [40]:
avg = df.groupby('product_name')['total_cost'].transform('mean')

In [41]:
df['avg'] = avg

In [42]:
df

,product_id,vendor_id,market_date,customer_id,quantity,cost_to_customer_per_qty,transaction_time,product_id,product_name,product_size,product_category_id,product_qty_type,total_cost,avg
0,1,7,2019-07-03,14,0.99,6.99,0 days 17:32:00,1,Habanero Peppers - Organic,medium,1,lbs,6.9201,16.120788
1,1,7,2019-07-03,14,2.18,6.99,0 days 18:23:00,1,Habanero Peppers - Organic,medium,1,lbs,15.2382,16.120788
2,1,7,2019-07-03,15,1.53,6.99,0 days 18:41:00,1,Habanero Peppers - Organic,medium,1,lbs,10.6947,16.120788
3,1,7,2019-07-03,16,2.02,6.99,0 days 18:18:00,1,Habanero Peppers - Organic,medium,1,lbs,14.1198,16.120788
4,1,7,2019-07-03,22,0.66,6.99,0 days 17:34:00,1,Habanero Peppers - Organic,medium,1,lbs,4.6134,16.120788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,16,4,2020-09-30,4,2.00,0.50,0 days 18:48:00,16,Sweet Corn,Ear,1,unit,1.0000,2.540638
4217,16,4,2020-09-30,11,1.00,0.50,0 days 16:30:00,16,Sweet Corn,Ear,1,unit,0.5000,2.540638
4218,16,4,2020-09-30,16,5.00,0.50,0 days 18:25:00,16,Sweet Corn,Ear,1,unit,2.5000,2.540638
4219,16,4,2020-09-30,18,6.00,0.50,0 days 18:28:00,16,Sweet Corn,Ear,1,unit,3.0000,2.540638


In [44]:
import numpy as np

In [ ]:
np.where()